In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "id": "36d2fcbc",
   "metadata": {},
   "source": [
    "## Integrating Flask with IBM Cloud"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "id": "40a673bc",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      " * Serving Flask app '__main__' (lazy loading)\n",
      " * Environment: production\n",
      "\u001b[31m   WARNING: This is a development server. Do not use it in a production deployment.\u001b[0m\n",
      "\u001b[2m   Use a production WSGI server instead.\u001b[0m\n",
      " * Debug mode: on\n"
     ]
    },
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.\n",
      " * Running on http://127.0.0.1:2020\n",
      "Press CTRL+C to quit\n",
      " * Restarting with stat\n"
     ]
    },
    {
     "ename": "SystemExit",
     "evalue": "1",
     "output_type": "error",
     "traceback": [
      "An exception has occurred, use %tb to see the full traceback.\n",
      "\u001b[1;31mSystemExit\u001b[0m\u001b[1;31m:\u001b[0m 1\n"
     ]
    }
   ],
   "source": [
    "#importing required libraries\n",
    "\n",
    "from flask import Flask, request, render_template\n",
    "import numpy as np\n",
    "import pandas as pd\n",
    "from sklearn import metrics \n",
    "import warnings\n",
    "import pickle\n",
    "import requests\n",
    "warnings.filterwarnings('ignore')\n",
    "from feature import FeatureExtraction\n",
    "\n",
    "file = open(\"model.pkl\",\"rb\")\n",
    "gbc = pickle.load(file)\n",
    "file.close()\n",
    "\n",
    "# NOTE: you must manually set API_KEY below using information retrieved from your IBM Cloud account.\n",
    "API_KEY = \"cWGD5yTjEpEGtqPpvHPDBElN5eXFS7eh2JRDyUWhySMW\"\n",
    "token_response = requests.post('https://iam.cloud.ibm.com/identity/token', data={\"apikey\":\n",
    " API_KEY, \"grant_type\": 'urn:ibm:params:oauth:grant-type:apikey'})\n",
    "mltoken = token_response.json()[\"access_token\"]\n",
    "\n",
    "header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}\n",
    "\n",
    "\n",
    "app = Flask(__name__)\n",
    "\n",
    "@app.route(\"/\", methods=[\"GET\", \"POST\"])\n",
    "def index():\n",
    "    if request.method == \"POST\":\n",
    "\n",
    "        url = request.form[\"url\"]\n",
    "        obj = FeatureExtraction(url)\n",
    "        x = np.array(obj.getFeaturesList()).reshape(1,30) \n",
    "\n",
    "        y_pred =gbc.predict(x)[0]\n",
    "        #1 is safe       \n",
    "        #-1 is unsafe\n",
    "        y_pro_phishing = gbc.predict_proba(x)[0,0]\n",
    "        y_pro_non_phishing = gbc.predict_proba(x)[0,1]\n",
    "        # if(y_pred ==1 ):\n",
    "        pred = \"It is {0:.2f} % safe to go \".format(y_pro_phishing*100)\n",
    "        payload_scoring = {\"input_data\": [{\"field\": [[\"UsingIP\",\"LongURL\",\"ShortURL\",\"Symbol@\",\"Redirecting//\",\"PrefixSuffix-\",\"SubDomains\",\"HTTPS\",\"DomainRegLen\",\"Favicon\",\"NonStdPort\",\"HTTPSDomainURL\",\"RequestURL\",\"AnchorURL\",\"LinksInScriptTags\",\"ServerFormHandler\",\"InfoEmail\",\"AbnormalURL\",\"WebsiteForwarding\",\"StatusBarCust\",\"DisableRightClick\",\"UsingPopupWindow\",\"IframeRedirection\",\"AgeofDomain\",\"DNSRecording\",\"WebsiteTraffic\",\"PageRank\",\"GoogleIndex\",\"LinksPointingToPage\",\"StatsReport\"\n",
    "]], \"values\": [[1,1,1,1,1,-1,-1,-1,-1,1,1,1,1,-1,-1,1,1,1,0,1,1,1,1,-1,-1,-1,-1,1,0,1]]}]}\n",
    "        response_scoring = requests.post('https://us-south.ml.cloud.ibm.com/ml/v4/deployments/084b5c52-f617-40ef-a0e8-3e6cf79ae447/predictions?version=2022-11-06', json=payload_scoring,\n",
    "        headers={'Authorization': 'Bearer ' + mltoken})\n",
    "        print(\"Scoring response\")\n",
    "        predictions=response_scoring.json()\n",
    "#print(predictions)\n",
    "        pred=print(predictions['predictions'][0]['values'][0][0])\n",
    "        return render_template('index.html',xx =round(y_pro_non_phishing,2),url=url )\n",
    "    return render_template(\"index.html\", xx =-1)\n",
    "    \n",
    "\n",
    "if __name__ == \"__main__\":\n",
    "    app.run(debug=True,port=2020)\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "c00b2ab7",
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}